In [2]:
import pandas as pd
from sklearn import tree
from sklearn.metrics import accuracy_score
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import itertools
import math
from TMDataset import TMDataset
import const
import util
import numpy as np


In [3]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import average_precision_score
from inspect import signature

In [4]:
df = pd.read_csv("TransportationData/datasetBalanced/dataset_5secondWindow.csv")

In [5]:
df

,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,...,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,...,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,Luca
2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,...,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Car,Luca
4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,...,89.770732,89.770732,89.770732,0.006389,9.000000,9.000000,9.000000,NaN,Car,andrea
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5888,14125,4.0,NaN,100.0,10.066610,4.564648,13.414389,3.373972,0.986319,0.974523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walking,IvanHeibi
5889,19582,32.0,NaN,10.0,9.964271,8.768007,11.454213,0.665529,0.648602,0.648343,...,NaN,NaN,NaN,NaN,31.960054,31.960054,31.960054,NaN,Train,Pierpaolo
5890,16178,86.0,NaN,100.0,9.812893,9.772881,9.862587,0.017633,0.024392,0.024081,...,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,Luca
5891,10973,199.0,NaN,100.0,9.831144,9.807347,9.874450,0.013485,0.481212,0.481209,...,NaN,NaN,NaN,NaN,1.090415,1.090415,1.090415,NaN,Still,Damiano


In [8]:
#df.columns

In [6]:
class TMDetection:
    dataset = TMDataset()
    classes = []
    classes2string = {
    }
    classes2number = {}

    def __init__(self):
        if not const.HAVE_DT:
            self.dataset.create_balanced_dataset(const.SINTETIC_LEARNING)
        classes_dataset = self.dataset.get_dataset['target'].values
        print(classes_dataset)
        for i, c in enumerate(sorted(set(classes_dataset))):
            self.classes2string[i] = c
            self.classes2number[c] = i
            self.classes.append(c)

    def __get_sets_for_classification(self, df_train, df_test, features):
        train, test = util.fill_nan_with_mean_training(df_train, df_test)
        #df_train.to_csv("df_train_class.csv")
        #df_test.to_csv("df_test_class.csv")
        
        train_features = train[features].values
        train_classes = [self.classes2number[c] for c in train['target'].values]
        test_features = test[features].values
        test_classes = [self.classes2number[c] for c in test['target'].values]
        return train_features, train_classes, test_features, test_classes

    # random forest algorithm training on training al train set and test on all test set
    def random_forest(self, sensors_set):
        features = list(self.dataset.get_sensors_set_features(sensors_set))
        print("RANDOM FOREST.....")
        print("CLASSIFICATION BASED ON THESE SENSORS: ", self.dataset.get_remained_sensors(sensors_set))
        print("NUMBER OF FEATURES: ", len(features))
        #print(features)
        train_features, train_classes, test_features, test_classes = self.__get_sets_for_classification(
            self.dataset.get_train, self.dataset.get_test, features)
        classifier_forest = RandomForestClassifier(n_estimators=const.PAR_RF_ESTIMATOR)
        classifier_forest.fit(train_features, train_classes)
        test_prediction = classifier_forest.predict(test_features)
        #print("train_features",train_features)
        # metrics
        acc = accuracy_score(test_classes, test_prediction)
        test_pred = test_prediction.tolist()
        #print(test_classes,test_prediction)
        precision = precision_score(test_classes, test_pred, average='macro')
        f1 = f1_score(test_classes, test_pred, average='macro')
        #recall = recall_score(test_classes, test_prediction)
                
        
        #df_feature = pd.DataFrame(
            #{'accuracy': acc, 'featureName': features, 'importance': classifier_forest.feature_importances_})
        #df_feature = df_feature.sort_values(by='importance', ascending=False)
        print("ACCURACY : " + str(acc), "Precision:" + str(precision),"F1 score:" + str(f1))
        
        print("END RANDOM FOREST")

        if not os.path.exists(const.DIR_RESULTS):
            os.makedirs(const.DIR_RESULTS)
        #df_feature.to_csv(const.DIR_RESULTS + "/" + str(sensors_set) + const.FILE_RANDOM_FOREST_RESULTS, index=False)


In [7]:
if __name__ == "__main__":
    detection = TMDetection()
    detection.random_forest(3)




START CREATE BALANCED DATASET....
./TransportationData/_Dataset/dataset.csv
END CREATE BALANCED DATASET....
['Bus' 'Bus' 'Bus' ... 'Walking' 'Walking' 'Walking']
RANDOM FOREST.....
CLASSIFICATION BASED ON THESE SENSORS:  ['android.sensor.game_rotation_vector', 'android.sensor.gyroscope', 'android.sensor.rotation_vector', 'android.sensor.gyroscope_uncalibrated', 'android.sensor.accelerometer', 'android.sensor.linear_acceleration', 'sound', 'android.sensor.orientation', 'speed']
NUMBER OF FEATURES:  36
['android.sensor.linear_acceleration#mean', 'android.sensor.rotation_vector#mean', 'sound#min', 'android.sensor.game_rotation_vector#max', 'sound#max', 'android.sensor.rotation_vector#std', 'android.sensor.accelerometer#std', 'android.sensor.orientation#min', 'speed#std', 'android.sensor.gyroscope#mean', 'android.sensor.orientation#std', 'android.sensor.rotation_vector#max', 'android.sensor.game_rotation_vector#std', 'speed#min', 'speed#mean', 'android.sensor.rotation_vector#min', 'android

In [8]:
df1 = pd.read_csv("df_train_class.csv")
df2 = pd.read_csv("df_test_class.csv")        

In [9]:
df1

,Unnamed: 0,time,activityrecognition#0,activityrecognition#1,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,...,speed#mean,speed#min,speed#max,speed#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,target,user
0,0,108,tilting,100.0,0.997513,0.997513,0.997513,0.000408,0.053704,0.015872,...,17.739895,17.739895,17.739895,0.628595,NaN,NaN,NaN,NaN,Car,U12
1,1,81,vehicle,100.0,0.916540,0.916328,0.916751,0.000299,0.071748,0.060176,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Train,U7
2,2,29,NaN,NaN,0.189758,0.187733,0.192763,0.001766,0.023559,0.006238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Train,U1
3,3,88,NaN,NaN,0.995498,0.995498,0.995498,0.004844,0.528280,0.203716,...,0.395000,0.390000,0.400000,0.007071,2341.0,2341.0,2341.0,6.363961,Walking,U2
4,4,458,vehicle,100.0,0.987923,0.987847,0.988177,0.000100,0.014080,0.002765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Train,U1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1827,1827,336,foot,80.0,0.668395,0.666302,0.669400,0.001448,0.094858,0.006476,...,0.000000,0.000000,0.000000,0.000000,31399.0,31399.0,31399.0,NaN,Walking,U3
1828,1828,326,vehicle,100.0,0.986401,0.985971,0.986724,0.000241,0.050070,0.013414,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Train,U1
1829,1829,134,foot,100.0,0.999505,0.999505,0.999505,0.002026,0.903823,0.319711,...,1.200000,1.200000,1.200000,NaN,2470.0,2470.0,2470.0,NaN,Walking,U2
1830,1830,59,NaN,NaN,0.039402,0.039031,0.039716,0.000187,0.001601,0.000000,...,0.000000,0.000000,0.000000,0.000000,28966.0,28966.0,28966.0,NaN,Still,U12


In [39]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)
# Fit the model on training set
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 

0.7874015748031497


/home/simple/anaconda3/envs/StriveSchool/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [40]:
filename = 'finalized_model.sav'
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

0.7874015748031497
